In [15]:
import pdbufr
import sys
import traceback
 
from math import isnan

from eccodes import *
from ecmwf.opendata import Client
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import numpy as np
import xarray as xr

import haversine as hs

import warnings
warnings.filterwarnings("ignore")

In [16]:
## CODE TRIALS WITH FORECAST OF THE 17th AUGUST ##

start_date_forecast = datetime(2023, 8, 17, 0)

In [17]:
## DOWNLOAD DATA FORECAST ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
client = Client(source="azure")

start_date = start_date_forecast.strftime("%Y%m%d")

client.retrieve(
    date=int(start_date),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target=f"data/tc_test_{start_date}.bufr",
);

20230817000000-240h-enfo-tf.bufr:   0%|          | 0.00/752k [00:00<?, ?B/s]

In [18]:
## LOAD TRACKS FORECAST DATA IN A DATAFRAME ##
# Function to import forecast storms file and load it in a dataframe
def create_storms_df():
    # Load cyclone dataframe with Mean sea level pressure value
    df_storms = pdbufr.read_bufr(f"data/tc_test_{start_date}.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    # Load cyclone dataframe with Wind speed at 10m value
    df1 = pdbufr.read_bufr(f"data/tc_test_{start_date}.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    # Add the Wind speed at 10m column to the storms dataframe 
    df_storms["windSpeedAt10M"] = df1.windSpeedAt10M
    # Storms with number higher than 10 are not real storms (according to what Fernando said)
    drop_condition = df_storms.stormIdentifier < '11'
    df_storms = df_storms[drop_condition]
    return df_storms

df_storms_forecast = create_storms_df()

## LOAD OBSERVED TRACKS IN A DATAFRAME
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

print('Observed tracks: ', df_storms_observed.NAME.squeeze().unique().tolist())
print('Forecasted tracks: ', df_storms_forecast.stormIdentifier.unique())

Observed tracks:  ['KHANUN', 'DORA', 'LAN', 'FERNANDA', 'GREG', 'HILARY']
Forecasted tracks:  ['07E' '08E' '09E' '09W']


In [19]:
## FIND THE CORRESPONDENT OBSERVED STORMS TO THOSE PRESENT IN THE FORECAST ##
for_storms = df_storms_forecast.stormIdentifier.unique()
obs_storms = df_storms_observed.NAME.squeeze().unique().tolist()

storms_pair = []

for f_storm in for_storms:
    dff = df_storms_forecast[df_storms_forecast.stormIdentifier == f_storm]
    loc_f = (dff.iloc[0].latitude, dff.iloc[0].longitude)
    max_dist = 40075 # kms of the equator
    for o_storm in obs_storms:
        dfo = df_storms_observed[df_storms_observed.NAME.squeeze() == o_storm]
        loc_o = (dfo.iloc[-1].LAT.squeeze(), dfo.iloc[-1].LON.squeeze())
        # Compute the distance between two point on earth with the haversine distance (output in km)
        hs_dist = hs.haversine(loc_f, loc_o)
        if hs_dist < max_dist:
            pair = f"{f_storm}-{o_storm}"
            max_dist = hs_dist
    storms_pair.append(pair)
    
print(storms_pair)

['07E-FERNANDA', '08E-GREG', '09E-HILARY', '09W-LAN']


In [20]:
## WIDGETS FOR INTERACTIVE MAPS PLOT ##

# Cyclone dropdown selection
cyclone = widgets.Dropdown(
    options = storms_pair,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Longitude slider
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options=longitude,
    index=(0, len(longitude)-1),
    description='Longitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

longitude_slider.style.description_width = '68px'
longitude_slider.style.handle_color = 'lightgreen'

# Latitude slider
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options=latitude,
    index=(0, len(latitude)-1),
    description='Latitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

latitude_slider.style.description_width = '58px'
latitude_slider.style.handle_color = 'lightgreen'

In [24]:
## INTERACTIVE MAP FOR TROPICAL CYCLONE TRACK ##

def plot_cyclone_tracks(cyclone, lat_boundaries, lon_boundaries):
    toplot = []
    
    # storm data preparation for plotting
    df_storms_forecast = create_storms_df()
    df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
    code, name = cyclone.split('-') 
    df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
    members = df_f.ensembleMemberNumber.unique()
    df_o = df_storms_observed[df_storms_observed.NAME.squeeze() == name]
    lat_o = df_o.LAT.squeeze().to_numpy(dtype='float')
    lon_o = df_o.LON.squeeze().to_numpy(dtype='float')
    
    # settings of the geographical area
    bottom, up = lat_boundaries
    left, right = lon_boundaries
    area = mmap(
        subpage_map_projection="cylindrical",
        subpage_lower_left_longitude=int(left),
        subpage_lower_left_latitude=int(bottom),
        subpage_upper_right_longitude=int(right),
        subpage_upper_right_latitude=int(up),
    )
    toplot.append(area)
    
    # settings of the coastline
    coast = mcoast(
        map_coastline_land_shade = "on",
        map_coastline_land_shade_colour = "cream",
        map_coastline_sea_shade = "on",
        map_coastline_sea_shade_colour = "#70CEE2",
        # map_cities = "on",
        map_grid_line_style = "dash",
        map_grid_colour = "black",
        map_label = "on",
        map_label_colour = "brown",
        map_coastline_colour = "brown",
    )
    toplot.append(coast)
    
    # settings of the tracks colour
    colours = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "brown"]
    colour = 0
    
    # Plot the ensemble tracks of the forecast
    for member in members:

        df_track = df_f[df_f.ensembleMemberNumber == member]
        df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)
        lon = df_track.longitude.to_numpy()
        lat = df_track.latitude.to_numpy()

        data = minput(
            input_type = 'geographical',
            input_x_values = lon,
            input_y_values = lat,
        )

        line=msymb(
            symbol_type='marker',
            symbol_marker_index = 28,
            symbol_colour = colours[colour],
            symbol_height = 0.20,
            symbol_text_font_colour = "black",       
            symbol_connect_line ='on'
        )

        colour += 1
        if colour == len(colours):
            colour = 0

        toplot.append(data)
        toplot.append(line)
    
    # Plot the observed track of the cyclone
    data_obs = minput(
        input_type = 'geographical',
        input_x_values = lon_o,
        input_y_values = lat_o,
    )
    toplot.append(data_obs)

    line_obs = msymb(
        symbol_type='marker',
        symbol_marker_index = 28,
        symbol_colour = "black",
        symbol_height = 0.20,
        symbol_text_font_colour = "black",       
        symbol_connect_line ='on'
    )
    toplot.append(line_obs)
    
    title = mtext(
        text_lines= [f"<font colour='navy'> <b> {cyclone} observed and forecasted trajectories </b> </font>",
                    # f"{start.strftime('%d %b %Y %H:%M')} - {end.strftime('%d %b %Y %H:%M')}"
                    ],
        text_justification= 'centre',
        text_font_size= 0.7,
        # text_font_style= 'bold',
        text_mode='title',
    )
    toplot.append(title)
    
    display(plot(toplot))

In [14]:
widgets.interactive(plot_cyclone_tracks, cyclone=cyclone, lat_boundaries=latitude_slider, lon_boundaries=longitude_slider)

interactive(children=(Dropdown(description='Active Storms:', index=3, options=('07E-FERNANDA', '08E-GREG', '09…

In [86]:
code, name = storms_pair[2].split('-')

df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
df_o = df_storms_observed[df_storms_observed.NAME.squeeze() == name]
initial_lat_lon = (df_f.latitude.iloc[0], df_f.longitude.iloc[0])

def forecast_tracks_locations(df_storm_forecast):
    members = df_storm_forecast.ensembleMemberNumber.unique()
    locations = []
    for member in members:
        df_track = df_f[df_f.ensembleMemberNumber == member]
        df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)
        latitude = df_track.latitude.tolist()
        longitude = df_track.longitude.tolist()
        locs = []
        for i in range(len(latitude)):
            loc = (latitude[i], longitude[i])
            locs.append(loc)
        locations.append(locs)
    return locations

def observed_track_locations(df_storm_observed):
    latitude = df_storm_observed.LAT.squeeze().tolist()
    longitude = df_storm_observed.LON.squeeze().tolist()
    locations = []
    for i in range(len(latitude)):
        loc = (latitude[i], longitude[i])
        locations.append(loc)
    return locations
        
locations_f = forecast_tracks_locations(df_f)
locations_o = observed_track_locations(df_o)

In [97]:
## INTERACTIVE MAP FOR TROPICAL CYCLONE TRACK ##
# Create the basemap for plotting
tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.0,
    scroll_wheel_zoom=True,
)

# Define observed tracks polyline element for the map
track_o = ipyleaflet.Polyline(
        locations=locations_o,
        color= "black",
        fill=False,
        weight=2,
    )

# Define forecasted tracks polyline element for the map
colours = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "brown"]
tracks_layer = []
markers_layer = []
colour = 0
for locs in locations_f:
    
    track = ipyleaflet.Polyline(
        locations=locs,
        color= colours[colour],
        fill=False,
        weight=2,
        # name='Track %.02d' % i,
    )
    
    markers = [ipyleaflet.CircleMarker(location=loc, radius=1, color=colours[colour]) for loc in locs]
    
    colour += 1
    if colour == len(colours):
        colour = 0
    
    marker_layer = ipyleaflet.LayerGroup(layers=markers)
    
    tracks_layer.append(track)
    markers_layer.append(marker_layer)

# Add observed track to the map
marker_o = [ipyleaflet.CircleMarker(location=loc, radius=1, color="black") for loc in locations_o]
markers_layer_o = ipyleaflet.LayerGroup(layers=marker_o)
layer_group_o = ipyleaflet.LayerGroup(layers=[track_o, markers_layer_o], name='Observed Track')

tc_track_map.add_layer(layer_group_o)

# Add forecasted ensemble tracks to the map
tracks_layer_group = ipyleaflet.LayerGroup(layers=tracks_layer)
markers_layer_group = ipyleaflet.LayerGroup(layers=markers_layer)
layer_group_f = ipyleaflet.LayerGroup(layers=[tracks_layer_group, markers_layer_group], name='Forecasted Ensemble Tracks')

tc_track_map.add_layer(layer_group_f)

# Add layers widget to the map
layers_control = ipyleaflet.LayersControl()
tc_track_map.add_control(layers_control);

# Print map
tc_track_map

Map(center=[13.6, -105.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…